In [60]:
import numpy as np
import pandas as pd
import datetime

import matplotlib
import matplotlib.pyplot as plt

In [61]:
df_account=pd.read_csv('account.asc', sep=';')
df_card=pd.read_csv('card.asc', sep=';')
df_client=pd.read_csv('client.asc', sep=';')
df_disp=pd.read_csv('disp.asc', sep=';')
df_district=pd.read_csv('district.asc', sep=';')
df_loan=pd.read_csv('loan.asc', sep=';')
df_order=pd.read_csv('order.asc', sep=';')
df_trans=pd.read_csv('trans.asc', sep=';')

C:\ISCTE3\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
print(df_card.head())
print(df_client.head())
print(df_disp.head())
print(df_district.head())
print(df_loan.head())
print(df_order.head())
print(df_trans.head())

   card_id  disp_id     type           issued
0     1005     9285  classic  931107 00:00:00
1      104      588  classic  940119 00:00:00
2      747     4915  classic  940205 00:00:00
3       70      439  classic  940208 00:00:00
4      577     3687  classic  940215 00:00:00
   client_id  birth_number  district_id
0          1        706213           18
1          2        450204            1
2          3        406009            1
3          4        561201            5
4          5        605703            5
   disp_id  client_id  account_id       type
0        1          1           1      OWNER
1        2          2           2      OWNER
2        3          3           2  DISPONENT
3        4          4           3      OWNER
4        5          5           3  DISPONENT
   A1           A2               A3       A4  A5  A6  A7  A8  A9    A10  \
0   1  Hl.m. Praha           Prague  1204953   0   0   0   1   1  100.0   
1   2      Benesov  central Bohemia    88884  80  26   6   2   5

In [63]:
df_trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN


In [64]:
df_trans.dtypes

trans_id        int64
account_id      int64
date            int64
type           object
operation      object
amount        float64
balance       float64
k_symbol       object
bank           object
account       float64
dtype: object

In [65]:
#"PRIJEM" stands for credit
#"VYDAJ" stands for withdrawal

print(df_trans['type'].value_counts())
aux = df_trans[(df_trans.type == 'VYDAJ')& (df_trans.operation == 'VYBER')]
print(aux.type.count())
print(aux.head(40))
###vyber type is the same as withdrawl

VYDAJ     634571
PRIJEM    405083
VYBER      16666
Name: type, dtype: int64
418252
     trans_id  account_id    date   type operation   amount  balance k_symbol  \
136    695560        2378  930131  VYDAJ     VYBER  34700.0  59453.7      NaN   
189    145127         485  930203  VYDAJ     VYBER  25300.0  46177.9      NaN   
222   1028004        3510  930206  VYDAJ     VYBER   1600.0  22271.0      NaN   
249    622121        2122  930207  VYDAJ     VYBER  14100.0  32859.1      NaN   
267    110785         374  930208  VYDAJ     VYBER   1500.0  21941.0      NaN   
293    393999        1338  930210  VYDAJ     VYBER   5200.0  33215.5      NaN   
301    695561        2378  930211  VYDAJ     VYBER  25200.0  85871.7      NaN   
309    615783        2099  930211  VYDAJ     VYBER  29700.0  52262.4      NaN   
324   1101121        3760  930212  VYDAJ     VYBER  14400.0  36059.2      NaN   
331   1549098        5270  930212  VYDAJ     VYBER   9600.0  36059.2      NaN   
348   1143204        3909 

In [66]:
def change_type(Ttype):
    if (Ttype == 'PRIJEM'):
        return 'CREDIT'
    else:
        return 'WITHDRAWAL'
df_trans['type']= df_trans['type'].map(change_type)

In [67]:
df_trans['operation'].value_counts()

VYBER             434918
PREVOD NA UCET    208283
VKLAD             156743
PREVOD Z UCTU      65226
VYBER KARTOU        8036
Name: operation, dtype: int64

In [68]:
'''
"VYBER KARTOU" credit card withdrawal
"VKLAD" credit in cash
"PREVOD Z UCTU" collection from another bank
"VYBER" withdrawal in cash
"PREVOD NA UCET" remittance to another bank
'''
def change_op(operation):
    if(operation == 'VYBER KARTOU'):
        return 'credit_card_withd'
    elif(operation == 'VKLAD'):
        return 'credit_cash'
    elif(operation == 'VYBER'):
        return 'withd_cash'
    elif(operation == 'PREVOD Z UCTU'):
        return 'collection_from_other_bank'
    else:
        return 'remittance_other_bank'
    

In [69]:
df_trans['operation']=df_trans['operation'].map(change_op)
print(df_trans.head(5))

   trans_id  account_id    date    type    operation  amount  balance  \
0    695247        2378  930101  CREDIT  credit_cash   700.0    700.0   
1    171812         576  930101  CREDIT  credit_cash   900.0    900.0   
2    207264         704  930101  CREDIT  credit_cash  1000.0   1000.0   
3   1117247        3818  930101  CREDIT  credit_cash   600.0    600.0   
4    579373        1972  930102  CREDIT  credit_cash   400.0    400.0   

  k_symbol bank  account  
0      NaN  NaN      NaN  
1      NaN  NaN      NaN  
2      NaN  NaN      NaN  
3      NaN  NaN      NaN  
4      NaN  NaN      NaN  


In [70]:
df_trans['k_symbol'].value_counts()

UROK           183114
SLUZBY         155832
SIPO           118065
                53433
DUCHOD          30338
POJISTNE        18500
UVER            13580
SANKC. UROK      1577
Name: k_symbol, dtype: int64

In [71]:
aux = df_trans[(df_trans.k_symbol !='UROK')&(df_trans.k_symbol !='SLUBZY')&(df_trans.k_symbol !='SIPO')&(df_trans.k_symbol !='DUCHOD')&(df_trans.k_symbol !='POJISTNE')&(df_trans.k_symbol !='UVER')&(df_trans.k_symbol !='SANKC. UROK')]
print(aux)

         trans_id  account_id    date        type          operation  amount  \
0          695247        2378  930101      CREDIT        credit_cash   700.0   
1          171812         576  930101      CREDIT        credit_cash   900.0   
2          207264         704  930101      CREDIT        credit_cash  1000.0   
3         1117247        3818  930101      CREDIT        credit_cash   600.0   
4          579373        1972  930102      CREDIT        credit_cash   400.0   
...           ...         ...     ...         ...                ...     ...   
1051814    606211        2063  981230  WITHDRAWAL  credit_card_withd  1400.0   
1051815    606741        2065  981230  WITHDRAWAL         withd_cash  4640.0   
1051816    713704        2439  981230  WITHDRAWAL         withd_cash   250.0   
1051817    713387        2437  981230  WITHDRAWAL         withd_cash  1470.0   
1051818    492431        1679  981230  WITHDRAWAL         withd_cash   840.0   

         balance k_symbol bank  account

In [72]:
'''
	"POJISTNE" stands for insurrance payment
"SLUZBY" stands for payment for statement
"UROK" stands for interest credited
"SANKC. UROK" sanction interest if negative balance
"SIPO" stands for household
"DUCHOD" stands for old-age pension
"UVER" stands for loan payment
'''
def change_k(k):
    if(k == 'POJISTNE'):
        return 'insurrance_payment'
    elif(k == 'SLUZBY'):
        return 'payment_for_statement'
    elif(k == 'UROK'):
        return 'interest_credited'
    elif(k == 'SANKC. UROK'):
        return 'sanction_negative_balance'
    elif(k == 'SIPO'):
        return 'household'
    elif(k == 'DUCHOD'):
        return 'old-age_pension'
    elif(k == 'UVER'):
        return 'loan_payment'
    else:
        return 'UNKNOWN'

In [73]:
df_trans['k_symbol']=df_trans['k_symbol'].map(change_k)

In [74]:
df_trans.head(20)

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,CREDIT,credit_cash,700.0,700.0,UNKNOWN,NaN,NaN
1,171812,576,930101,CREDIT,credit_cash,900.0,900.0,UNKNOWN,NaN,NaN
2,207264,704,930101,CREDIT,credit_cash,1000.0,1000.0,UNKNOWN,NaN,NaN
3,1117247,3818,930101,CREDIT,credit_cash,600.0,600.0,UNKNOWN,NaN,NaN
4,579373,1972,930102,CREDIT,credit_cash,400.0,400.0,UNKNOWN,NaN,NaN
5,771035,2632,930102,CREDIT,credit_cash,1100.0,1100.0,UNKNOWN,NaN,NaN
6,452728,1539,930103,CREDIT,credit_cash,600.0,600.0,UNKNOWN,NaN,NaN
7,725751,2484,930103,CREDIT,credit_cash,1100.0,1100.0,UNKNOWN,NaN,NaN
8,497211,1695,930103,CREDIT,credit_cash,200.0,200.0,UNKNOWN,NaN,NaN
9,232960,793,930103,CREDIT,credit_cash,800.0,800.0,UNKNOWN,NaN,NaN


In [75]:
df_account.head(10)

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102
5,1972,77,POPLATEK MESICNE,930102
6,1539,1,POPLATEK PO OBRATU,930103
7,793,47,POPLATEK MESICNE,930103
8,2484,74,POPLATEK MESICNE,930103
9,1695,76,POPLATEK MESICNE,930103


In [76]:
df_account.frequency.value_counts()

POPLATEK MESICNE      4167
POPLATEK TYDNE         240
POPLATEK PO OBRATU      93
Name: frequency, dtype: int64

In [77]:
'''
"POPLATEK MESICNE" stands for monthly issuance
"POPLATEK TYDNE" stands for weekly issuance
"POPLATEK PO OBRATU" stands for issuance after transaction
'''

def change_freq(freq):
    if(freq=='POPLATEK MESICNE'):
        return 'monthly'
    if(freq=='POPLATEK TYDNE'):
        return 'weekly'
    else:
        return 'after_trasaction'


In [79]:
df_account['frequency'] = df_account['frequency'].map(change_freq)
df_account= df_account.rename(columns={"frequency": "issuance_frequency"}) 
df_account.head(10)

,account_id,district_id,issuance_frequency,date
0,576,55,monthly,930101
1,3818,74,monthly,930101
2,704,55,monthly,930101
3,2378,16,monthly,930101
4,2632,24,monthly,930102
5,1972,77,monthly,930102
6,1539,1,after_trasaction,930103
7,793,47,monthly,930103
8,2484,74,monthly,930103
9,1695,76,monthly,930103
